<a href="https://colab.research.google.com/github/shreya-rawal/notebooks/blob/main/webscraping_financial_data_from_wikipedia_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
from bs4 import BeautifulSoup
import html5lib
import requests
import pandas as pd

## Extract Data Using Web Scraping

The wikipedia webpage https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks provides information about largest banks in the world by various parameters. Scrape the data from the table 'By market capitalization' and store it in a JSON file.

### Webpage Contents

Gather the contents of the webpage in text format using the `requests` library and assign it to the variable <code>html_data</code>


In [69]:
url = 'https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks'

In [70]:
html_data = requests.get(url).text #get the contents of the webpage in text format

In [71]:
soup = BeautifulSoup(html_data, "html.parser")

In [94]:
tables = soup.find_all('table')

4

In [107]:
required_table = soup.find_all("table", attrs={"class": "wikitable"})[2]

In [115]:
print(required_table.prettify())

<table class="wikitable sortable mw-collapsible">
 <tbody>
  <tr>
   <th data-sort-type="number">
    Rank
   </th>
   <th>
    Bank name
   </th>
   <th>
    Market cap
    <br/>
    (US$ billion)
   </th>
  </tr>
  <tr>
   <td>
    1
   </td>
   <td>
    <span class="flagicon">
     <a href="/web/20200318083015/https://en.wikipedia.org/wiki/United_States" title="United States">
      <img alt="United States" class="thumbborder" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20200318083015im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20200318083015im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20200318083015im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_

In [151]:
data = pd.DataFrame(columns=["Bank name", "Market cap (US$ billion)"])

for row in required_table.tbody.find_all('tr'):
  col = row.find_all('td')
  if (col != []):
    bank_name = col[1].text.split('\n')
    market_cap = col[2].text.split()
    data = data.append({"Bank name":bank_name[0], "Market cap (US$ billion)": round(float(market_cap[0]), 2)}, ignore_index=True)

<ipython-input-151-c8bcf5ad8bd0>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Bank name":bank_name[0], "Market cap (US$ billion)": round(float(market_cap[0]), 2)}, ignore_index=True)
<ipython-input-151-c8bcf5ad8bd0>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Bank name":bank_name[0], "Market cap (US$ billion)": round(float(market_cap[0]), 2)}, ignore_index=True)
<ipython-input-151-c8bcf5ad8bd0>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Bank name":bank_name[0], "Market cap (US$ billion)": round(float(market_cap[0]), 2)}, ignore_index=True)
<ipython-input-151-c8bcf5ad8bd0>:8: FutureWarning: The frame.append method is deprecated and will be removed 

In [163]:
#data.index +=1
#shifted the index to start with 1
data.index.name = "Rank"

In [164]:
data.head(10)

,Bank name,Market cap (US$ billion)
Rank,,
1,JPMorgan Chase,390.93
2,Industrial and Commercial Bank of China,345.21
3,Bank of America,325.33
4,Wells Fargo,308.01
5,China Construction Bank,257.40
6,HSBC Holdings PLC,219.27
7,Agricultural Bank of China,203.24
8,Citigroup Inc.,203.16
9,Bank of China,181.47


In [165]:
#loading the result dataframe into json file
data.to_json("by_market_capitalization.json")